## pyfixest examples

In [20]:
# create some data

import pandas as pd
import numpy as np
from pyfixest.api import feols

# create data
np.random.seed(123)
N = 100000
k = 4
G = 25
X = np.random.normal(0, 1, N * k).reshape((N,k))
X = pd.DataFrame(X)
X[1] = np.random.choice(list(range(0, 50)), N, True)
X[2] = np.random.choice(list(range(0, 1000)), N, True)
X[3] = np.random.choice(list(range(0, 1000)), N, True)

beta = np.random.normal(0,1,k)
beta[0] = 0.005
u = np.random.normal(0,1,N)
Y = 1 + X @ beta + u
cluster = np.random.choice(list(range(0,G)), N)

Y = pd.DataFrame(Y)
Y.rename(columns = {0:'Y'}, inplace = True)
X = pd.DataFrame(X)

data = pd.concat([Y, X], axis = 1)
data.rename(columns = {0:'X1', 1:'X2', 2:'X3', 3:'X4'}, inplace = True)
data['X4'] = data['X4'].astype('category')
data['X3'] = data['X3'].astype('category')
data['X2'] = data['X2'].astype('category')
data['group_id'] = cluster
data['Y2'] = data.Y + np.random.normal(0, 1, N)

In [14]:
# HC1 inference
feols('Y ~ X1 | X2 + X3 + X4', 'hetero', data)

[  depvar colnames      coef        se     tstat    pvalue
 0      Y       X1  0.001469  0.003164  0.464335  0.642408]

In [15]:
# HC3 inference
feols('Y ~ X1 | X2 + X3 + X4', 'HC3', data)

[  depvar colnames      coef        se     tstat    pvalue
 0      Y       X1  0.001469  0.003164  0.464344  0.642401]

In [17]:
# CRV1
feols(fml = 'Y ~ X1', vcov = {'CRV1':'group_id'}, data = data)

[  depvar   colnames        coef        se       tstat    pvalue
 0      Y  Intercept -577.090042  1.094118 -527.447656  0.000000
 1      Y         X1    1.389563  1.023390    1.357804  0.174526]

In [16]:
# CRV1
feols(fml = 'Y ~ X1', vcov = {'CRV3':'group_id'}, data = data)

[  depvar   colnames        coef        se       tstat    pvalue
 0      Y  Intercept -577.090042  1.139483 -506.449086  0.000000
 1      Y         X1    1.389563  1.066219    1.303261  0.192486]

In [19]:
#  multiple estimation = multiple dependend variables

feols(fml = 'Y + Y2 ~ X1 | X3 + X4 ', vcov = 'hetero', data = data)

[  depvar colnames      coef       se     tstat    pvalue
 0      Y       X1  0.018823  0.01947  0.966804  0.333642,
   depvar colnames      coef        se     tstat    pvalue
 0     Y2       X1  0.015098  0.019733  0.765114  0.444204]